In [23]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
import time
import tables
from scipy.io import loadmat

In [24]:
camParams = np.load('../Calib/secondCalib/camParams.npy')
dist_coeffs = np.load('../Calib/secondCalib/distCoeff.npy')
point1 = cv2.imread('../Tests/secondTest/point1.jpg')
point2 = cv2.imread('../Tests/secondTest/point2.jpg')
point3 = cv2.imread('../Tests/secondTest/point3.jpg')
point4 = cv2.imread('../Tests/secondTest/point4.jpg')
point5 = cv2.imread('../Tests/secondTest/point5.jpg')
point6 = cv2.imread('../Tests/secondTest/point6.jpg')

In [25]:
camParams

array([[512.10571572,   0.        ,   0.        ],
       [  0.        , 512.29967519,   0.        ],
       [324.93693113, 242.11127231,   1.        ]])

In [266]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 4, 1.0)
obj_points = np.array([[0.1, 0.065, 0], [0, 0.065, 0], [0.1, 0, 0], [0, 0, 0]])
dist_coeffs= np.array([-0.405925403776238, 0.206430617505333, 0, 0, 0], dtype = np.float32)
cameraMatrix = camParams
def centerExt(frame):
    LED=np.zeros((4,2))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nonzero = cv2.findNonZero(gray)
    nonzero = np.array(nonzero).reshape(nonzero.shape[0], 2)
    nonzero = np.float32(nonzero)
    ret,label,center = cv2.kmeans(nonzero,4,None,criteria,4,cv2.KMEANS_RANDOM_CENTERS)
    origin = ((center[0,0]+center[1,0]+center[2,0]+center[3,0])/4, (center[0,1]+center[1,1]+center[2,1]+center[3,1])/4)
    for j in range(4):
        LED[j]=center[j]-origin
    LED_lab=np.array([[LED[(LED[:, 0]>0) & (LED[:, 1]>0)][0][0]+origin[0],LED[(LED[:, 0]>0) & (LED[:, 1]>0)][0][1]+origin[1]],
                      [LED[(LED[:, 0]<0) & (LED[:, 1]>0)][0][0]+origin[0],LED[(LED[:, 0]<0) & (LED[:, 1]>0)][0][1]+origin[1]],
                      [LED[(LED[:, 0]>0) & (LED[:, 1]<0)][0][0]+origin[0],LED[(LED[:, 0]>0) & (LED[:, 1]<0)][0][1]+origin[1]],
                      [LED[(LED[:, 0]<0) & (LED[:, 1]<0)][0][0]+origin[0],LED[(LED[:, 0]<0) & (LED[:, 1]<0)][0][1]+origin[1]]])
    #cv2.circle(frame,(int(LED_lab[0, 0]), int(LED_lab[0, 1])), 5, (255,255,255), -1)
    #cv2.imshow('t', frame)
    #cv2.waitKey(0)
    return LED_lab

In [267]:
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rotationMatrixToEulerAngles(R) :
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

In [268]:
LED_lab2 = centerExt(point1)
LED_lab3 = centerExt(point6)

In [269]:
LED_lab2

array([[484.73638916, 318.30960083],
       [275.95812988, 321.52557373],
       [476.27102661, 185.36915588],
       [276.37380981, 193.07475281]])

In [270]:
obj_points

array([[0.1  , 0.065, 0.   ],
       [0.   , 0.065, 0.   ],
       [0.1  , 0.   , 0.   ],
       [0.   , 0.   , 0.   ]])

In [271]:
testpts = np.array(LED_lab2).reshape(4, 1, 2)
testpts.shape

(4, 1, 2)

In [272]:
pts_test = cv2.undistortPoints(testpts, cameraMatrix, dist_coeffs.astype(np.float32))

In [289]:
ret2, rvec2, tvec2 = cv2.solvePnP(obj_points, LED_lab2.reshape(4, 1, 2), cameraMatrix, dist_coeffs, 5)
dst2, jac2 = cv2.Rodrigues(rvec2)
ret3, rvec3, tvec3 = cv2.solvePnP(obj_points, LED_lab3.reshape(4, 1, 2), cameraMatrix, dist_coeffs, 5)
dst3, jac3 = cv2.Rodrigues(rvec3)

In [290]:
np.linalg.norm(rvec3)*180/math.pi - np.linalg.norm(rvec2)*180/math.pi

-6.606661248027808

In [291]:
euler_flipped2 = rotationMatrixToEulerAngles(dst2)
euler2 = np.flip(euler_flipped2)
euler_flipped3 = rotationMatrixToEulerAngles(dst3)
euler3 = np.flip(euler_flipped3)

In [292]:
print(euler2*180/math.pi)
print(euler3*180/math.pi)

[-2.27329611 -8.52873305  7.49612156]
[ 0.02735115 -2.42299391  4.21042285]


In [293]:
diff_euler = (euler3-euler2)
diff_euler

array([ 0.04015387,  0.10656525, -0.05734626])

In [294]:
r_diff=np.matmul(dst3, dst2.transpose())

In [295]:
r_diff_euler = np.flip(rotationMatrixToEulerAngles(r_diff))*180/math.pi

In [296]:
r_diff_euler

array([ 1.82051795,  6.21778852, -3.05662294])